# Forecasting Prices using Traditional Supervised Learning Algorithms 

In [2]:
import numpy as np 
import pandas as pd 

In [3]:
data = pd.read_csv(r'C:\Users\cl501_27\Documents\GitHub\ForEx\processed_data.csv',index_col='Unnamed: 0')
data.head()

,Date,Open_price,Day_high,Day_low,Closing_price,Currency Pair,Trend_Open_price,Seasonal_Open_price,Residual_Open_price,Trend_Closing_price,...,Seasonal_Day_low,Residual_Day_low,RSI,MACD,MACD_Signal,BB_Upper,BB_Lower,day_of_week,month,is_weekend
0,2014-11-07,0.418073,0.415411,0.420199,0.418141,USD/INR,0.430544,0.482937,0.45972,0.430893,...,0.532825,0.464686,0.644914,0.494863,0.468996,0.371949,0.429018,0.666667,0.909091,0.0
1,2014-11-10,0.418829,0.415513,0.420199,0.418794,USD/INR,0.430544,0.503235,0.45972,0.430893,...,0.520792,0.464686,0.644914,0.495321,0.469106,0.371949,0.429018,0.000000,0.909091,0.0
2,2014-11-11,0.419035,0.414969,0.421339,0.418883,USD/INR,0.430544,0.489535,0.45972,0.430893,...,0.534282,0.464686,0.644914,0.495736,0.469295,0.371949,0.429018,0.166667,0.909091,0.0
3,2014-11-12,0.418884,0.415003,0.420268,0.418080,USD/INR,0.430544,0.493794,0.45972,0.430893,...,0.484180,0.464686,0.644914,0.495489,0.469387,0.371949,0.429018,0.333333,0.909091,0.0
4,2014-11-13,0.417922,0.415431,0.420268,0.419274,USD/INR,0.430544,0.489704,0.45972,0.430893,...,0.516613,0.464686,0.644914,0.496125,0.469614,0.371949,0.429018,0.500000,0.909091,0.0


In [4]:
data.columns

Index(['Date', 'Open_price', 'Day_high', 'Day_low', 'Closing_price',
       'Currency Pair', 'Trend_Open_price', 'Seasonal_Open_price',
       'Residual_Open_price', 'Trend_Closing_price', 'Seasonal_Closing_price',
       'Residual_Closing_price', 'Trend_Day_high', 'Seasonal_Day_high',
       'Residual_Day_high', 'Trend_Day_low', 'Seasonal_Day_low',
       'Residual_Day_low', 'RSI', 'MACD', 'MACD_Signal', 'BB_Upper',
       'BB_Lower', 'day_of_week', 'month', 'is_weekend'],
      dtype='object')

## Separating Data for each unique currency pair 

In [5]:
import pandas as pd

def split_data_by_currency_pair(data, currency_column='Currency Pair'):
    """
    Splits the main DataFrame into separate DataFrames for each unique value 
    in the currency_column, renaming columns to include the currency pair.

    Parameters:
    - data (pd.DataFrame): The main DataFrame with data for multiple currency pairs.
    - currency_column (str): The name of the column representing currency pairs.

    Returns:
    - dict: A dictionary with keys as unique currency pairs and values as separate DataFrames.
    """
    currency_dfs = {}
    
    for i in data[currency_column].unique():
        # Create a DataFrame for each currency pair and store in the dictionary
        df = data[data[currency_column] == i].copy()
        
        # Rename columns by appending the currency pair name
        df.columns = [f"{col}_{i}" if col != currency_column else col for col in df.columns]
        
        # Store the DataFrame in the dictionary with the currency pair as the key
        currency_dfs[i] = df

    return currency_dfs


# Call the function
"""currency_dfs = split_data_by_currency_pair(data)
currency_dfs"""


'currency_dfs = split_data_by_currency_pair(data)\ncurrency_dfs'

# 1. Random Forest

In [6]:
# Defining a function to plot the results of the model for each currency pair 


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *


rf = RandomForestRegressor() # Constructor of the model 
random_forest_mae = {}

def random_forest_model():
    # Train Test split the data based on unique currency pair 
    for i in data['Currency Pair'].unique():
        currency_dfs= split_data_by_currency_pair(data)
        df = currency_dfs[i]
        #print(df.columns)
        print(f"----------------------------------------------Working on Currency Pair : {i} now---------------------------------------------------")
        X = df.drop(['Currency Pair',f'Closing_price_{i}',f'Date_{i}'],axis=1)
        y= df[f'Closing_price_{i}']
        X_train , X_test , y_train,y_test = train_test_split(X,y,test_size=0.3)
        rf.fit(X_train,y_train)
        y_pred = rf.predict(X_test)
        ac = mean_absolute_error(y_test, y_pred)
        print(f'The mean absolute error for {i} is : {ac*100} %')
        random_forest_mae[i] = ac
    print(random_forest_mae)

random_forest_model()
        
    


----------------------------------------------Working on Currency Pair : USD/INR now---------------------------------------------------
The mean absolute error for USD/INRis : 0.06090985825484736 %
----------------------------------------------Working on Currency Pair : EUR/USD now---------------------------------------------------
The mean absolute error for EUR/USDis : 0.0001702775978847033 %
----------------------------------------------Working on Currency Pair : GBP/USD now---------------------------------------------------
The mean absolute error for GBP/USDis : 0.0002219769058585591 %
----------------------------------------------Working on Currency Pair : USD/JPY now---------------------------------------------------
The mean absolute error for USD/JPYis : 0.023360177663969105 %
----------------------------------------------Working on Currency Pair : EUR/INR now---------------------------------------------------
The mean absolute error for EUR/INRis : 0.08572042317545979 %
-----

In [13]:
# Step 1: Define the helper functions
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# Data splitting helper function
def split_data_by_currency_pair(data, currency_column='Currency Pair'):
    currency_dfs = {}
    for i in data[currency_column].unique():
        df = data[data[currency_column] == i].copy()
        df.columns = [f"{col}_{i}" if col != currency_column else col for col in df.columns]
        currency_dfs[i] = df
    return currency_dfs

# Data preparation function
def prepare_data_for_currency(df, currency_pair, target_column='Closing_price', date_column='Date', test_size=0.3, random_state=42):
    X = df.drop([f'{target_column}_{currency_pair}', f'{date_column}_{currency_pair}', 'Currency Pair'], axis=1)
    y = df[f'{target_column}_{currency_pair}']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test



In [15]:
#Step 2: Define the model functions
# Random Forest Model
def random_forest_model(data):
    currency_dfs = split_data_by_currency_pair(data)  # Make sure currency_dfs is defined
    rf = RandomForestRegressor()
    rf_metrics = {}
    
    for i in data['Currency Pair'].unique():
        df = currency_dfs[i]
        print(f"--- Working on Currency Pair: {i} ---")
        
        # Prepare data for training and testing
        X_train, X_test, y_train, y_test = prepare_data_for_currency(df, i)
        
        # Fit the model
        rf.fit(X_train, y_train)
        
        # Predict and calculate MAE
        y_pred = rf.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"Mean Absolute Error for {i}: {mae}")
        print(f"Mean Absolute Percentage Error (MAPE) for {i}: {mape * 100:.2f}%")
        print(f"R-squared (R^2) for {i}: {r2:.2f}")
        
        rf_metrics[i] = {'MAE': mae, 'MAPE': mape * 100, 'R^2': r2}
    
        print("\nRandom Forest Metrics for each currency pair:", rf_metrics)
        
        
       


# Step 3: Call the models with the main data
# Assuming 'data' is the main DataFrame you're working with
random_forest_model(data)



--- Working on Currency Pair: USD/INR ---
Mean Absolute Error for USD/INR: 0.0006380297161732671
Mean Absolute Percentage Error (MAPE) for USD/INR: 0.13%
R-squared (R^2) for USD/INR: 1.00

Random Forest Metrics for each currency pair: {'USD/INR': {'MAE': 0.0006380297161732671, 'MAPE': 0.1315202253744807, 'R^2': 0.9996394711746183}}
--- Working on Currency Pair: EUR/USD ---
Mean Absolute Error for EUR/USD: 1.8119892589465008e-06
Mean Absolute Percentage Error (MAPE) for EUR/USD: 0.05%
R-squared (R^2) for EUR/USD: 1.00

Random Forest Metrics for each currency pair: {'USD/INR': {'MAE': 0.0006380297161732671, 'MAPE': 0.1315202253744807, 'R^2': 0.9996394711746183}, 'EUR/USD': {'MAE': 1.8119892589465008e-06, 'MAPE': 0.04532631810051208, 'R^2': 0.999844627457715}}
--- Working on Currency Pair: GBP/USD ---
Mean Absolute Error for GBP/USD: 2.0972806810175094e-06
Mean Absolute Percentage Error (MAPE) for GBP/USD: 0.04%
R-squared (R^2) for GBP/USD: 1.00

Random Forest Metrics for each currency pa

#### 2. SVM

In [17]:
# SVM Model
def svm_model(data):
    currency_dfs = split_data_by_currency_pair(data)
    svm = SVR()
    svm_metrics = {}

    for i in data['Currency Pair'].unique():
        df = currency_dfs[i]
        print(f"--- Working on Currency Pair: {i} ---")
        
        # Prepare data
        X_train, X_test, y_train, y_test = prepare_data_for_currency(df, i)
        
        # Train the model
        svm.fit(X_train, y_train)
        
        # Predict
        y_pred = svm.predict(X_test)
        
        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"Mean Absolute Error for {i}: {mae}")
        print(f"Mean Absolute Percentage Error (MAPE) for {i}: {mape * 100:.2f}%")
        print(f"R-squared (R^2) for {i}: {r2:.2f}")
        
        svm_metrics[i] = {'MAE': mae, 'MAPE': mape * 100, 'R^2': r2}
    
    print("\nSVM Metrics for each currency pair:", svm_metrics)


svm_model(data)

--- Working on Currency Pair: USD/INR ---
Mean Absolute Error for USD/INR: 0.03934590641669505
Mean Absolute Percentage Error (MAPE) for USD/INR: 7.97%
R-squared (R^2) for USD/INR: -0.00
--- Working on Currency Pair: EUR/USD ---
Mean Absolute Error for EUR/USD: 0.00028476347003694675
Mean Absolute Percentage Error (MAPE) for EUR/USD: 6.75%
R-squared (R^2) for EUR/USD: -0.00
--- Working on Currency Pair: GBP/USD ---
Mean Absolute Error for GBP/USD: 0.000355216783294136
Mean Absolute Percentage Error (MAPE) for GBP/USD: 6.56%
R-squared (R^2) for GBP/USD: -0.01
--- Working on Currency Pair: USD/JPY ---
Mean Absolute Error for USD/JPY: 0.05538079177452392
Mean Absolute Percentage Error (MAPE) for USD/JPY: 7.09%
R-squared (R^2) for USD/JPY: 0.33
--- Working on Currency Pair: EUR/INR ---
Mean Absolute Error for EUR/INR: 0.04019907725850403
Mean Absolute Percentage Error (MAPE) for EUR/INR: 7.27%
R-squared (R^2) for EUR/INR: -0.03
--- Working on Currency Pair: JPY/INR ---
Mean Absolute Error 

##### hyperparameter tuning of SVM

In [20]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

def svm_model_with_hyperparameter_tuning(data):
    # Function to split the data by currency pairs (this needs to be defined)
    currency_dfs = split_data_by_currency_pair(data)
    
    # Initialize the SVM model
    svm = SVR()
    svm_metrics = {}
    
    # Define the hyperparameter grid to search over
    param_grid = {
        'C': [0.1, 1, 10, 100],  # Regularization parameter
        'kernel': ['linear', 'poly', 'rbf'],  # Kernel types
        'gamma': ['scale', 'auto', 0.1, 1],  # Kernel coefficient
        'epsilon': [0.1, 0.2, 0.3],  # Epsilon parameter in the loss function
    }

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
    
    # Iterate over each currency pair
    for currency_pair in data['Currency Pair'].unique():
        df = currency_dfs[currency_pair]
        print(f"--- Working on Currency Pair: {currency_pair} ---")
        
        # Prepare data (This should return X_train, X_test, y_train, y_test for each currency pair)
        X_train, X_test, y_train, y_test = prepare_data_for_currency(df, currency_pair)
        
        # Perform GridSearchCV to find the best hyperparameters
        grid_search.fit(X_train, y_train)
        
        # Best hyperparameters
        best_params = grid_search.best_params_
        print(f"Best Hyperparameters for {currency_pair}: {best_params}")
        
        # Get the best model
        best_svm = grid_search.best_estimator_
        
        # Make predictions with the best model
        y_pred = best_svm.predict(X_test)
        
        # Calculate evaluation metrics
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # Convert to percentage
        r2 = r2_score(y_test, y_pred)
        
        # Store the metrics for each currency pair
        svm_metrics[currency_pair] = {
            'MAE': mae,
            'MAPE': mape,
            'R^2': r2
        }
        
        print(f"MAE for {currency_pair}: {mae}")
        print(f"MAPE for {currency_pair}: {mape}%")
        print(f"R^2 for {currency_pair}: {r2}")
    
    # Return the metrics for all currency pairs
    return svm_metrics
  
svm_model_with_hyperparameter_tuning(data)

--- Working on Currency Pair: USD/INR ---
Best Hyperparameters for USD/INR: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
MAE for USD/INR: 0.03934590641669505
MAPE for USD/INR: 7.97333126922707%
R^2 for USD/INR: -0.0007951072480518739
--- Working on Currency Pair: EUR/USD ---
Best Hyperparameters for EUR/USD: {'C': 0.1, 'epsilon': 0.3, 'gamma': 'scale', 'kernel': 'linear'}
MAE for EUR/USD: 0.0002847634700369464
MAPE for EUR/USD: 6.7507843150736395%
R^2 for EUR/USD: -0.0019961656071314504
--- Working on Currency Pair: GBP/USD ---
Best Hyperparameters for GBP/USD: {'C': 0.1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
MAE for GBP/USD: 0.000355216783294136
MAPE for GBP/USD: 6.562801109365292%
R^2 for GBP/USD: -0.01122850326968683
--- Working on Currency Pair: USD/JPY ---
Best Hyperparameters for USD/JPY: {'C': 1, 'epsilon': 0.1, 'gamma': 0.1, 'kernel': 'poly'}
MAE for USD/JPY: 0.050490530981156856
MAPE for USD/JPY: 6.453541740382089%
R^2 for USD/JPY: 0.4299919

{'USD/INR': {'MAE': 0.03934590641669505,
  'MAPE': 7.97333126922707,
  'R^2': -0.0007951072480518739},
 'EUR/USD': {'MAE': 0.0002847634700369464,
  'MAPE': 6.7507843150736395,
  'R^2': -0.0019961656071314504},
 'GBP/USD': {'MAE': 0.000355216783294136,
  'MAPE': 6.562801109365292,
  'R^2': -0.01122850326968683},
 'USD/JPY': {'MAE': 0.050490530981156856,
  'MAPE': 6.453541740382089,
  'R^2': 0.4299919412868569},
 'EUR/INR': {'MAE': 0.04019907725850403,
  'MAPE': 7.27042876967689,
  'R^2': -0.02677561930424721},
 'JPY/INR': {'MAE': 0.0003248764341407288,
  'MAPE': 441156469370.09875,
  'R^2': -0.02687896175044857},
 'GBP/INR': {'MAE': 0.03723831068974665,
  'MAPE': 5.826684656911221,
  'R^2': 0.11239367074056628}}

JPY/INR has an exceptionally high MAPE, which likely indicates an issue with the model's predictions for this currency pair. The error may be due to variations.
USD/JPY shows the best R^2 value of 0.42999, indicating it is the most predictive pair among those tested, but it's still far from perfect (1.0).
GBP/USD performs well across MAE and MAPE, and its R^2 is negative but relatively closer to 0 compared to some other pairs.

#### 3. Gradient Boosting

In [36]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

def gradient_boosting_model(data):
    currency_dfs = split_data_by_currency_pair(data)
    gb = GradientBoostingRegressor()
    gb_metrics = {}

    for i in data['Currency Pair'].unique():
        df = currency_dfs[i]
        print(f"--- Working on Currency Pair: {i} ---")
        
        # Prepare data
        X_train, X_test, y_train, y_test = prepare_data_for_currency(df, i)
        
        # Train the model
        gb.fit(X_train, y_train)
        
        # Predict
        y_pred = gb.predict(X_test)
        
        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"Mean Absolute Error for {i}: {mae}")
        print(f"Mean Absolute Percentage Error (MAPE) for {i}: {mape * 100:.2f}%")
        print(f"R-squared (R^2) for {i}: {r2:.2f}")
        
        gb_metrics[i] = {'MAE': mae, 'MAPE': mape * 100, 'R^2': r2}
    
    print("\nGradient Boosting Metrics for each currency pair:", gb_metrics)
   

gradient_boosting_model(data)

--- Working on Currency Pair: USD/INR ---
Mean Absolute Error for USD/INR: 0.000565094379370444
Mean Absolute Percentage Error (MAPE) for USD/INR: 0.12%
R-squared (R^2) for USD/INR: 1.00
--- Working on Currency Pair: EUR/USD ---
Mean Absolute Error for EUR/USD: 3.3070496922511585e-06
Mean Absolute Percentage Error (MAPE) for EUR/USD: 0.08%
R-squared (R^2) for EUR/USD: 1.00
--- Working on Currency Pair: GBP/USD ---
Mean Absolute Error for GBP/USD: 3.7422471459227727e-06
Mean Absolute Percentage Error (MAPE) for GBP/USD: 0.07%
R-squared (R^2) for GBP/USD: 1.00
--- Working on Currency Pair: USD/JPY ---
Mean Absolute Error for USD/JPY: 0.00046171357480649514
Mean Absolute Percentage Error (MAPE) for USD/JPY: 0.06%
R-squared (R^2) for USD/JPY: 1.00
--- Working on Currency Pair: EUR/INR ---
Mean Absolute Error for EUR/INR: 0.0009498070454154396
Mean Absolute Percentage Error (MAPE) for EUR/INR: 0.17%
R-squared (R^2) for EUR/INR: 1.00
--- Working on Currency Pair: JPY/INR ---
Mean Absolute Er

#### 4. KNN



In [35]:
from sklearn.neighbors import KNeighborsRegressor


def knn_model(data):
    currency_dfs = split_data_by_currency_pair(data)
    knn = KNeighborsRegressor()
    knn_metrics = {}

    for i in data['Currency Pair'].unique():
        df = currency_dfs[i]
        print(f"--- Working on Currency Pair: {i} ---")
        
        # Prepare data
        X_train, X_test, y_train, y_test = prepare_data_for_currency(df, i)
        
        # Train the model
        knn.fit(X_train, y_train)
        
        # Predict
        y_pred = knn.predict(X_test)
        
        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"Mean Absolute Error for {i}: {mae}")
        print(f"Mean Absolute Percentage Error (MAPE) for {i}: {mape * 100:.2f}%")
        print(f"R-squared (R^2) for {i}: {r2:.2f}")
        
        knn_metrics[i] = {'MAE': mae, 'MAPE': mape * 100, 'R^2': r2}
    
    print("\nKNN Metrics for each currency pair:", knn_metrics)
    
 
knn_model(data)

--- Working on Currency Pair: USD/INR ---
Mean Absolute Error for USD/INR: 0.006605630324972945
Mean Absolute Percentage Error (MAPE) for USD/INR: 1.36%
R-squared (R^2) for USD/INR: 0.97
--- Working on Currency Pair: EUR/USD ---
Mean Absolute Error for EUR/USD: 0.00030464107455757493
Mean Absolute Percentage Error (MAPE) for EUR/USD: 7.25%
R-squared (R^2) for EUR/USD: -0.15
--- Working on Currency Pair: GBP/USD ---
Mean Absolute Error for GBP/USD: 0.0003473075619520789
Mean Absolute Percentage Error (MAPE) for GBP/USD: 6.36%
R-squared (R^2) for GBP/USD: 0.04
--- Working on Currency Pair: USD/JPY ---
Mean Absolute Error for USD/JPY: 0.017228340903452904
Mean Absolute Percentage Error (MAPE) for USD/JPY: 2.16%
R-squared (R^2) for USD/JPY: 0.91
--- Working on Currency Pair: EUR/INR ---
Mean Absolute Error for EUR/INR: 0.010260429060206355
Mean Absolute Percentage Error (MAPE) for EUR/INR: 1.88%
R-squared (R^2) for EUR/INR: 0.92
--- Working on Currency Pair: JPY/INR ---
Mean Absolute Error

In [38]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ----- ---------------------------------- 17.6/124.9 MB 92.0 MB/s eta 0:00:02
   --------- ------------------------------ 29.4/124.9 MB 74.4 MB/s eta 0:00:02
   ------------- -------------------------- 41.2/124.9 MB 67.0 MB/s eta 0:00:02
   ------------------ --------------------- 57.4/124.9 MB 68.9 MB/s eta 0:00:01
   ------------------ --------------------- 59.0/124.9 MB 67.1 MB/s eta 0:00:01
   -------------------- ------------------- 65.0/124.9 MB 51.8 MB/s eta 0:00:02
   ------------------------ --------------- 76.0/124.9 MB 51.6 MB/s eta 0:00:01
   ------------------------------ --------- 94.9/124.9 MB 57.1 MB/s eta 0:00:01
   -------------------------------- ------ 104.3/124.9 MB 55.5 MB/s eta 0:00:01
   ------------------------------------ -- 116.4/124.9 MB 55.5 MB/s eta 0:00:01
   --------------------------------------  124.8/12

DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 5. xgboost

In [41]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score

def xgboost_model(data):
    # Split the data by currency pair
    currency_dfs = split_data_by_currency_pair(data)
    xgb = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=6)
    xgb_metrics = {}

    for i in data['Currency Pair'].unique():
        df = currency_dfs[i]
        print(f"--- Working on Currency Pair: {i} ---")
        
        # Prepare data
        X_train, X_test, y_train, y_test = prepare_data_for_currency(df, i)
        
        # Train the model
        xgb.fit(X_train, y_train)
        
        # Predict
        y_pred = xgb.predict(X_test)
        
        # Calculate metrics
        mae = mean_absolute_error(y_test, y_pred)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        print(f"Mean Absolute Error for {i}: {mae}")
        print(f"Mean Absolute Percentage Error (MAPE) for {i}: {mape * 100:.2f}%")
        print(f"R-squared (R^2) for {i}: {r2:.2f}")
        
        xgb_metrics[i] = {'MAE': mae, 'MAPE': mape * 100, 'R^2': r2}
    
    print("\nXGBoost Metrics for each currency pair:", xgb_metrics)
    


xgboost_model(data)


--- Working on Currency Pair: USD/INR ---
Mean Absolute Error for USD/INR: 0.000512993702697205
Mean Absolute Percentage Error (MAPE) for USD/INR: 0.10%
R-squared (R^2) for USD/INR: 1.00
--- Working on Currency Pair: EUR/USD ---
Mean Absolute Error for EUR/USD: 2.0205778504925016e-05
Mean Absolute Percentage Error (MAPE) for EUR/USD: 0.49%
R-squared (R^2) for EUR/USD: 0.99
--- Working on Currency Pair: GBP/USD ---
Mean Absolute Error for GBP/USD: 2.2884629774914292e-05
Mean Absolute Percentage Error (MAPE) for GBP/USD: 0.42%
R-squared (R^2) for GBP/USD: 0.99
--- Working on Currency Pair: USD/JPY ---
Mean Absolute Error for USD/JPY: 0.000443389325900292
Mean Absolute Percentage Error (MAPE) for USD/JPY: 0.05%
R-squared (R^2) for USD/JPY: 1.00
--- Working on Currency Pair: EUR/INR ---
Mean Absolute Error for EUR/INR: 0.0008851933734354613
Mean Absolute Percentage Error (MAPE) for EUR/INR: 0.16%
R-squared (R^2) for EUR/INR: 1.00
--- Working on Currency Pair: JPY/INR ---
Mean Absolute Erro